In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed, interact_manual
from ipywidgets import IntSlider, IntRangeSlider, FloatSlider, interactive, Layout, Dropdown, IntText, HBox, VBox, Output
from IPython.display import display, clear_output
import random
import scipy.signal
from math import log
import array
from scipy.signal import upfirdn, welch, decimate,convolve
from scipy.special import erfc
import time
print("Libraries added successfully!")

Libraries added successfully!


In [2]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

def generate_qam_constellation(L):
    M = L * L
    l = int(np.log2(L))
    
    # Generate the QAM constellation points
    x = np.arange(-(L - 1), L, 2)
    y = np.arange(-(L - 1), L, 2)
    xv, yv = np.meshgrid(x, y)
    mapping = xv + 1j * yv
    mapping = mapping.flatten()

    return mapping

def plot_qam_constellation(L):
    # Start timer
    loader_html3.value = loading
    start_time = time.time()

    mapping = generate_qam_constellation(L)
    M = L * L
    l = int(np.log2(L))

    # Plotting the constellation
    plt.figure(figsize=(10, 7))
    plt.scatter(mapping.real, mapping.imag)
    if L < 16:  # Include text for points if L is less than 16
        # Gray code labels
        labels = [bin(i)[2:].zfill(2 * l) for i in range(M)]
        dx, dy = -0.5, 0.3  # Label offsets
        for i in range(len(labels)):
            plt.text(mapping[i].real + dx, mapping[i].imag + dy, labels[i], bbox=dict(facecolor='red', alpha=0.5))
    
    plt.grid(True)
    plt.xlim(-1 * L, 1 * L)
    plt.ylim(-1 * L, 1 * L)
    plt.title(f"{L}x{L} QAM Constellation")
    plt.xlabel("In-phase")
    plt.ylabel("Quadrature")
    plt.show()
    
    # Show elapsed time
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html3.value = done

# Dropdown for selecting L
L_dropdown = Dropdown(options=[2, 4, 8, 16, 32, 64], value=8, description='L-QAM:', continuous_update=False)

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create interactive widget
interactive_plot = interact(plot_qam_constellation, L=L_dropdown)

input_widgets = VBox([L_dropdown], layout=Layout(width='auto'))
plot_output = interactive_plot.widget.children[-1]  # The output plot

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox that includes both the input widgets and the loading animation
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs_and_loader, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)

In [3]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

def generate_psk_constellation(M):
    k = int(np.log2(M))
    ph1 = np.pi / 4

    # Initialize theta
    theta = np.array([ph1, -ph1, np.pi - ph1, -np.pi + ph1])
    mapping = np.exp(1j * theta)

    # Generate PSK constellation
    if k > 2:
        for j in range(3, k + 1):
            theta = theta / 2
            mapping = np.exp(1j * theta)
            mapping = np.concatenate((mapping, -np.conjugate(mapping)))
            theta = np.real(np.log(mapping) / 1j)

    return mapping

def plot_psk_constellation(M):
    # Start timer
    loader_html3.value = loading
    start_time = time.time()

    constellation = generate_psk_constellation(M)
    k = int(np.log2(M))
    plt.figure(figsize=(10, 7))
    plt.scatter(np.real(constellation), np.imag(constellation))
    plt.grid(True)
    plt.title(f'{M}-PSK Constellation')
    plt.xlabel('In-Phase')
    plt.ylabel('Quadrature')
    plt.axhline(0, color='gray', linewidth=0.5)
    plt.axvline(0, color='gray', linewidth=0.5)
    for m in range(len(constellation)):
        plt.text(np.real(constellation[m]) + 0.05, np.imag(constellation[m]), 
                 format(m, '0{}b'.format(k)), 
                 bbox=dict(facecolor='red', alpha=0.5))
    plt.show()

    # Show elapsed time
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html3.value = done

# Create a dropdown widget
M_dropdown = widgets.Dropdown(
    options=[4, 8, 16, 32, 64],
    value=16,
    description='M-PSK:'
)

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create interactive widget
interactive_plot = interact(plot_psk_constellation, M=M_dropdown)

input_widgets = VBox([M_dropdown], layout=Layout(width='auto'))
plot_output = interactive_plot.widget.children[-1]  # The output plot

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox that includes both the input widgets and the loading animation
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs_and_loader, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)

In [4]:
# HTML loading animation and done message
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(value=loading)
timer_html3 = widgets.HTML(value="Elapsed time: - seconds")

# Function to calculate roll-off factor and recommend the best M
def calculate_rolloff(f1, f2, R):
    W = (f2 - f1) * (10 ** 6)
    R = R * (10 ** 6)
    possible_Ms = [4, 16, 64, 256, 1024, 4096, 16384]  # Possible values of M
    best_M = None
    best_a = None
    for M in possible_Ms:
        a = np.log2(M) * W / R - 1
        if 0 < a < 1:
            best_M = M
            best_a = a
            break
    return best_a, best_M

# Widgets for inputs
f1_input = widgets.FloatText(description='F1 (MHz):', value=6.75)
f2_input = widgets.FloatText(description='F2 (MHz):', value=9.25)
R_input = widgets.FloatText(description='R (Mbps):', value=12)

# Function to update the result
def update_result(f1, f2, R):
    # Start the timer
    loader_html3.value = loading
    start_time = time.time()

    a, M = calculate_rolloff(f1, f2, R)
    
    # Stop the timer and update the timer HTML
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    
    # Update the loading animation to done
    loader_html3.value = done
    
    # Display the result
    if M is not None:
        result_html.value = f"Result: a = {a:.4f}, Recommended M = {M}"
    else:
        result_html.value = "No suitable M found within the given constraints."

# Create interactive widget
interactive_widget = interactive(update_result, f1=f1_input, f2=f2_input, R=R_input)

# Result display
result_html = widgets.HTML(value="Result: -")

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Group the input widgets and the loader timer box together
inputs_and_loader = HBox([VBox([f1_input, f2_input, R_input], layout=Layout(width='auto')), loader_timer_box], layout=Layout(align_items='center'))

# Create a VBox that includes both the input widgets and the result display
ui = VBox([inputs_and_loader, result_html])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)

In [5]:
# Loading animation and done checkmark
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

warning_html = widgets.HTML(
    value=""
)

def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Br = 1
    Fs = Br * nsamp
    Td = 1 / Br
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off * roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted)
            h.append(factor1 * (factor2 + factor3))

    return h

def ber_qam(EbNo, M, roll_off, F1, F2, Br):
    F1 = F1 * 1e6  # Convert MHz to Hz
    F2 = F2 * 1e6  # Convert MHz to Hz
    Br = Br * 1e6  # Convert Mbps to Hz
    W = F2 - F1  # Bandwidth in Hz
    fc = F1 + W / 2  # Carrier frequency
    nsamp = int(np.ceil(2 * F2 / Br)) +7  # Number of samples per symbol

    a = 0.25
    L = int(np.sqrt(M))
    l = np.log2(L)
    k = 2 * l
    Nsymb = 10000
    SNR = EbNo - 10 * np.log10(nsamp / k / 2)  # σε db
    core = [1+1j, 1-1j, -1+1j, -1-1j]
    mapping = core[:]
    if l > 1:
        for j in range(1, int(l)):
            mapping = list(map(lambda x: x + j * 2 * core[0], mapping))
            conj_arr = np.conj(mapping)
            mapping = mapping + conj_arr.tolist()
            conj_arr = -np.conj(mapping)
            mapping = mapping + conj_arr.tolist()

    # Generate random sequence
    x = np.floor(2 * np.random.rand(int(k * Nsymb), 1))
    x_temp = np.reshape(x, (int(len(x) / (k)), int(k)))
    xsym = []

    # Split the list into sublists and put the contents of each sublist
    # into a string so that with the int() command it is converted from binary to decimal
    for i in range(len(x_temp)):
        my_str = ''
        y = x_temp[i]
        for j in range(int(np.log2(M))):
            my_str = my_str + str(int(y[j]))
        a = int(my_str, 2)
        xsym = xsym + [a]

    y = []
    for n in range(len(xsym)):
        y = y + [mapping[xsym[n]]]

    delay = 10
    filtorder = delay * nsamp * 2

    shaping_filter = rootRaisedCosine(nsamp, roll_off, delay)
    ytx = upfirdn([1], y, nsamp)  # upsample
    ytx = np.convolve(ytx, shaping_filter)
    m = np.arange(1, len(ytx) + 1)
    s = np.real(np.multiply(ytx, np.exp(1j * 2 * np.pi * fc * m / nsamp)))

    s_matrix = np.matrix(s)  # transpose
    s_matrix = s_matrix.getH()
    s_list = s_matrix.tolist()
    Ps = 10 * np.log10(np.matmul(s, s_list) / len(s))  # Power of complex signal in dB
    Pn = Ps - SNR

    n = np.sqrt(10**(Pn / 10)) * np.random.randn(1, len(ytx))
    snoisy = s + n

    # receiver
    yrx = 2 * np.multiply(snoisy, np.exp(-1j * 2 * np.pi * fc * m / nsamp))
    yrx = yrx[0, :]
    yrx = np.convolve(yrx, shaping_filter)
    yrx = yrx[::nsamp]  # downsample

    yrx = yrx[2 * delay + 0:len(yrx) - 2 * delay]

    yi = yrx.copy()
    yq = np.imag(yi)
    yi = np.real(yi)

    xrx = []
    q = np.arange(-L + 1, L, 2)

    for n in range(len(yrx)):
        differences = np.abs(q - yi[n])  # Array with the differences of the signal from the levels
        m = min(differences)
        [index], = np.where(differences == m)
        yi[n] = q[index]
        differences = np.abs(q - yq[n])  # Array with the differences of the signal from the levels
        m = min(differences)
        [index], = np.where(differences == m)
        yq[n] = q[index]
    error = 0
    for i in range(len(yrx)):
        if y[i] != yi[i] + yq[i] * 1j:
            error += 1 
    return error / len(x)

def plot_ber_qam(M, roll_off, F1, F2, Br):
    if F1 >= F2:
        warning_html.value = "<div style='color: red; font-size: 16px;'>Warning: F1 should be less than F2.</div>"
        return
    
    warning_html.value = ""  # Clear any previous warnings
    
    # Start the timer
    loader_html3.value = loading
    start_time = time.time()

    ber_exp = []
    ber_th = []
    L = int(np.sqrt(M))
    for i in range(1, 15):
        ber_exp.append(ber_qam(i, M, roll_off, F1, F2, Br))
        ber_th.append(((L - 1) /(L*np.log2(L)) * scipy.special.erfc(np.sqrt(3 * np.log2(L) / (L * L - 1) * 10**(i/10)))))

    plt.figure(figsize=(10, 8))
    plt.semilogy(range(1, 15), ber_th)  # Plot theoretical BER as a line
    plt.semilogy(range(1, 15), ber_exp, 'o')  # Plot experimental BER as points
    plt.legend(['Theoretical', 'Simulation'])
    plt.xlabel('Eb/N0(db)')
    plt.ylabel('Bit Error Probability')
    plt.title(f'BER curve for {M}-QAM')
    plt.grid(which='both')
    plt.show()

    # Stop the timer and update the timer HTML
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    # Update the loading animation to done
    loader_html3.value = done

# Define QAM options
qam_options = {'4-QAM': 4, '16-QAM': 16, '64-QAM': 64}
qam_selector = widgets.Dropdown(options=qam_options, value=16, description='QAM Type:')

# Define additional input boxes for roll-off, F1, F2, and Br
roll_off_input = widgets.FloatText(value=0.25, description='Roll-off:')
F1_input = widgets.FloatText(value=6.75, description='F1: (MHz)')
F2_input = widgets.FloatText(value=9.25, description='F2: (MHz)')
Br_input = widgets.FloatText(value=10, description='Br: (Mbps)')

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create interactive widget
interactive_plot = interactive(plot_ber_qam, M=qam_selector, roll_off=roll_off_input, F1=F1_input, F2=F2_input, Br=Br_input)

input_widgets = VBox([qam_selector, roll_off_input, F1_input, F2_input, Br_input, warning_html], layout=Layout(width='auto'))
plot_output = interactive_plot.children[-1]  # The output plot

# Create a VBox that includes both the input widgets and the loading animation
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs_and_loader, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)

In [6]:
def rtrapezium(nsamp, rolloff, delay):
    T = 1
    t = np.arange(-delay*T, (delay*T) + 1/nsamp, 1/nsamp)
    rrc = np.zeros_like(t)
    for i, ti in enumerate(t):
        if ti == 0.0:
            rrc[i] = 1.0 - rolloff + 4*rolloff/np.pi
        elif abs(ti) == T / (4 * rolloff):
            rrc[i] = (rolloff / np.sqrt(2)) * (((1 + 2/np.pi) * (np.sin(np.pi / (4 * rolloff)))) + ((1 - 2/np.pi) * (np.cos(np.pi / (4 * rolloff)))))
        else:
            rrc[i] = (np.sin(np.pi * ti * (1 - rolloff) / T) + 4 * rolloff * ti * np.cos(np.pi * ti * (1 + rolloff) / T)) / (np.pi * ti * (1 - (4 * rolloff * ti / T) ** 2))
    return rrc / np.sqrt(np.sum(rrc**2))

def run_simulation(f1, f2, qam_type):
    with output8:
        clear_output(wait=True)

        # Parameters
        k = int(np.log2(qam_type))
        M = 2**k
        Nsymb = 30000
        pulse_type = 1  # 1 for rtrapezium shaping filter, 0 for rectangular pulse
        nsamp = 32  # oversampling factor
        fc = (f1 + f2) / 2  # carrier frequency
        bandwidth = f2 - f1  # signal bandwidth
        rolloff = bandwidth / (2 * fc)  # adjust rolloff factor based on bandwidth
        EbNo = 10  # Eb/No in dB
        SNR = EbNo - 10 * np.log10(nsamp / k / 2)  # SNR per signal sample

        # Phase and mapping initialization
        ph1 = np.pi / 4
        theta = np.array([ph1, -ph1, np.pi - ph1, -np.pi + ph1])
        mapping = np.exp(1j * theta)

        if k > 2:
            for j in range(3, k + 1):
                theta = theta / 2
                mapping = np.exp(1j * theta)
                mapping = np.concatenate([mapping, -np.conj(mapping)])
                theta = np.angle(mapping)

        # Transmitter
        x = np.random.randint(0, 2, k * Nsymb)  # random binary sequence
        xsym = x.reshape(-1, k)
        xsym = xsym.dot(2**np.arange(xsym.shape[-1])[::-1])  # bitwise to decimal
        y = mapping[xsym]

        # Shaping filter definition
        if pulse_type == 1:  # Nyquist pulse -- rtrapezium
            delay = 8  # Group delay (# of T periods)
            shaping_filter = rtrapezium(nsamp, rolloff, delay)
        else:  # Rectangular pulse
            delay = 0.5
            shaping_filter = np.ones(nsamp) / np.sqrt(nsamp)  # with normalization

        # Transmitted signal
        ytx = upfirdn([1], y, nsamp)
        ytx = convolve(ytx, shaping_filter, mode='same')

        # Quadrature modulation
        m = np.arange(len(ytx))
        s = np.real(ytx * np.exp(1j * 2 * np.pi * fc * m / nsamp))

        # Adding white Gaussian noise
        Ps = 10 * np.log10(np.mean(s**2))  # signal power in dB
        Pn = Ps - SNR  # corresponding noise power in dB
        n = np.sqrt(10**(Pn / 10)) * np.random.randn(len(ytx))
        snoisy = s + n  # noisy bandpass signal

        # Receiver
        yrx = 2 * snoisy * np.exp(-1j * 2 * np.pi * fc * m / nsamp)
        yrx = convolve(yrx, shaping_filter, mode='same')

        # Spectrum plot of received signal
        f, Pxx_den = welch(np.real(s), fs=nsamp, nperseg=1024)
        Pxx_den = 10 * np.log10(Pxx_den)
        plt.figure(figsize=(10, 8))
        plt.plot(f, Pxx_den, 'r')
        plt.title('Welch Power Spectral Density Estimate')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power Spectral Density (V^2/Hz)')
        plt.xlim(0, max(f1, f2) + 5)
        plt.grid()
        plt.show()

# Widgets for f1, f2, and QAM type
f1_widget = widgets.FloatText(value=6.75, description='f1:')
f2_widget = widgets.FloatText(value=9.25, description='f2:')
qam_widget = widgets.Dropdown(options=[4, 16, 64], value=16, description='QAM Type:')

def on_value_change(change):
    run_simulation(f1_widget.value, f2_widget.value, qam_widget.value)

f1_widget.observe(on_value_change, names='value')
f2_widget.observe(on_value_change, names='value')
qam_widget.observe(on_value_change, names='value')

output8 = widgets.Output()

display(qam_widget,f1_widget, f2_widget, output8)

# Run initial simulation
run_simulation(f1_widget.value, f2_widget.value, qam_widget.value)

Dropdown(description='QAM Type:', index=1, options=(4, 16, 64), value=16)

FloatText(value=6.75, description='f1:')

FloatText(value=9.25, description='f2:')

Output()

In [7]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# Function to calculate the maximum achievable transmission rate R' and the percentage increase
def calculate_R_and_percentage_increase(M, a, R):
    W = 2.5 * (10 ** 6)  # Fixed bandwidth in Hz
    log2M = np.log2(M)
    R_prime = (log2M * W) / (1 + a)  # Maximum achievable rate in bps
    R_prime_mbps = R_prime / (10 ** 6)  # Convert to Mbps
    percentage_increase = ((R_prime_mbps - R) / R) * 100
    return R_prime_mbps, percentage_increase

# Widgets for inputs
R_input = widgets.FloatText(description='R (Mbps):', value=8.0)
M_input = widgets.FloatText(description='M:', value=16)
a_input = widgets.FloatText(description='α\' (roll-off):', value=0.125)


# Output widget
output_vals = widgets.Output()

# Function to update the result
def update_result(change=None):
    # Start the timer
    loader_html3.value = loading
    start_time = time.time()

    with output_vals:
        output_vals.clear_output()
        R = R_input.value
        M = M_input.value
        a = a_input.value
        R_prime_mbps, percentage_increase = calculate_R_and_percentage_increase(M, a, R)
        print(f"Maximum Achievable Rate (R') = {R_prime_mbps:.3f} Mbps")
        print(f"Percentage Increase = {percentage_increase:.2f}%")

        # Stop the timer and update the timer HTML
        elapsed_time = time.time() - start_time
        timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
        # Update the loading animation to done
        loader_html3.value = done

# Observers to update the result when any input changes
R_input.observe(update_result, names='value')
M_input.observe(update_result, names='value')
a_input.observe(update_result, names='value')


# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create interactive widget
input_widgets = VBox([R_input, M_input, a_input], layout=Layout(width='auto'))

# Group the input widgets and the loader timer box together
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create a VBox that includes both the input widgets and the output
ui = VBox([inputs_and_loader, output_vals])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)

# Initial calculation
update_result()


In [8]:
# HTML for loading animation and completion checkmark
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
    </div>
    """

loader_html3 = widgets.HTML(value=loading)
timer_html3 = widgets.HTML(value="Elapsed time: - seconds")

# Define the root raised cosine filter function
def rootRaisedCosine1(nsamp, roll_off, delay):
    t = np.arange(-delay, delay + 1 / nsamp, 1 / nsamp)
    h = np.zeros(len(t))
    for i in range(len(t)):
        if t[i] == 0.0:
            h[i] = 1.0 - roll_off + 4 * roll_off / np.pi
        elif roll_off != 0 and t[i] == 1 / (4 * roll_off):
            h[i] = roll_off / np.sqrt(2) * ((1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off)))
        elif roll_off != 0 and t[i] == -1 / (4 * roll_off):
            h[i] = roll_off / np.sqrt(2) * ((1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off)))
        else:
            h[i] = (np.sin(np.pi * t[i] * (1 - roll_off)) + 4 * roll_off * t[i] * np.cos(np.pi * t[i] * (1 + roll_off))) / (np.pi * t[i] * (1 - (4 * roll_off * t[i]) ** 2))
    return h

# Define the BER computation functions
def compute_ber_psk(EbNo_dB, M1):
    EbNo_linear = 10**(EbNo_dB / 10)
    if M1 == 2:  # BPSK
        return 0.5 * scipy.special.erfc(np.sqrt(EbNo_linear))
    else:  # M-PSK
        k = np.log2(M1)
        return (1/4*k) * scipy.special.erfc(np.sqrt(EbNo_linear * k) * np.sin(np.pi / M1))

def ber_psk_simulation(EbNo_dB, M1):
    Nsymb = 30000  # Number of symbols
    nsamp = 16  # Samples per symbol
    fc = 4  # Carrier frequency
    rolloff = 0.25
    delay = 10
    SNR_dB = EbNo_dB - 10 * np.log10(nsamp / np.log2(M1))
    shaping_filter = rootRaisedCosine1(nsamp, rolloff, delay)
    filtorder = delay * nsamp * 2

    # Generate random bit stream
    bits1 = np.random.randint(0, M1, Nsymb)

    # Map bits to PSK symbols
    symbols = np.exp(1j * (2 * np.pi * bits1 / M1))

    # Upsample and filter
    ytx1 = upfirdn([1], symbols, nsamp)
    ytx1 = np.convolve(ytx1, shaping_filter, mode='same')
    m1 = np.arange(len(ytx1))
    s1 = np.real(ytx1 * np.exp(1j * 2 * np.pi * fc * m1 / nsamp))

    Ps = np.mean(np.abs(s1)**2)
    SNR_linear = 10**(SNR_dB / 10)
    Pn = Ps / SNR_linear
    noise = np.sqrt(Pn / 4) * (np.random.randn(len(s1)) + 1j * np.random.randn(len(s1)))
    snoisy = s1 + noise

    # Receiver
    yrx1 = snoisy * np.exp(-1j * 2 * np.pi * fc * m1 / nsamp)
    yrx1 = np.convolve(yrx1, shaping_filter, mode='same')
    yrx1 = yrx1[::nsamp]

    # Demodulate symbols
    detected_bits1 = np.angle(yrx1) * M1 / (2 * np.pi)
    detected_bits1 = np.round(detected_bits1) % M1

    # Calculate BER
    bit_errors1 = np.sum(bits1 != detected_bits1)
    ber1 = bit_errors1 / len(bits1)
    return ber1

def plot_ber_psk(M):
    # Start the timer
    loader_html3.value = loading
    start_time = time.time()

    ber_exp = []
    ber_th = []
    for i in range(1, 18):
        ber_exp.append(ber_psk_simulation(i, M))
        ber_th.append(compute_ber_psk(i, M))

    plt.figure(figsize=(10, 8))
    plt.semilogy(range(1, 18), ber_th)  # Plot theoretical BER as a line
    plt.semilogy(range(1, 18), ber_exp, 'o')  # Plot experimental BER as points
    plt.legend(['Theoretical', 'Simulation'])
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Probability')
    plt.title(f'BER curve for {M}-PSK')
    plt.grid(which='both')
    plt.show()

    # Stop the timer and update the timer HTML
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    # Update the loading animation to done
    loader_html3.value = done

psk_options = {'BPSK': 2, 'QPSK': 4, '8-PSK': 8}
psk_selector = widgets.Dropdown(options=psk_options, value=4, description='PSK Type:')

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create interactive widget
interactive_plot = interactive(plot_ber_psk, M=psk_selector)

input_widgets = VBox([psk_selector], layout=Layout(width='auto'))
plot_output = interactive_plot.children[-1]  # The output plot

# Create a VBox that includes both the input widgets and the loading animation
inputs_and_loader = HBox([input_widgets, loader_timer_box], layout=Layout(align_items='center'))

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs_and_loader, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)